In [ ]:
# This is a script for installing required packages

# using Pkg
# Pkg.add("Plots")
# Pkg.add("Distributions")
# Pkg.add("QuadGK")
# Pkg.add("SpecialFunctions")
# Pkg.add("Optim")
# Pkg.add("Printf")
# Pkg.add("DataFrames")
# Pkg.add("CSV")
# Pkg.add("Plots")
# Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/Origin.jl.git"))
# Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/Deformula.jl.git"))
# Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/SparseMatrix.jl.git"))
# Pkg.add(PackageSpec(url="https://github.com/JuliaReliab/NMarkov.jl.git"))

In [ ]:
using SparseArrays
using NMarkov
using Plots
using Distributions
using QuadGK
using SpecialFunctions
using Optim
using Printf
using DataFrames
using CSV
using SparseMatrix

In [ ]:
import NMarkov.eye
function eye(M::AbstractMatrix)
    eye(size(M)[1])
end

## Zhao and Nakagawa

In [ ]:
function GN(n, D0, D1)
    k = size(D0)[1]
    Qn = spzeros(AbstractMatrix{Float64}, n, n) # blockmatrix
    I = eye(D0)
    b = spzeros(AbstractMatrix{Float64}, n, 1)
    for i = 1:n
        Qn[i,i] = D0
    end
    for i = 1:n-1
        Qn[i,i+1] = D1
    end
    a = zeros(k, n*k)
    a[1:k,1:k] = I
    for i = 1:n
        b[i,1] = I
    end
    a, sparse(block(Qn)), Matrix(sparse(block(b)))
end

function gN(n, D0, D1)
    k = size(D0)[1]
    Qn = spzeros(AbstractMatrix{Float64}, n, n) # blockmatrix
    I = eye(D0)
    for i = 1:n
        Qn[i,i] = D0
    end
    for i = 1:n-1
        Qn[i,i+1] = D1
    end
    a = zeros(k, n*k)
    a[1:k,1:k] = I
    b = zeros(n*k, k)
    b[((n-1)*k)+1:(n*k),1:k] = D1
    a, sparse(block(Qn)), b
end

In [ ]:
function PfProb(t0, n, D0, D1, dist)
    a, A, b = GN(n, D0, D1)
    p1 = a * mexpmix(A, b, bounds = (0.0, t0), rmax = 10000) do t
        pdf(dist, t)
    end
    p3 = a * mexp(A, b, t0, rmax = 10000)

    a, A, b = gN(n, D0, D1)
    _, tmp = mexpcmix(A, b, bounds = (0.0, t0), rmax = 10000) do t
        pdf(dist, t)
    end
    p2 = a * tmp
    _, tmp = mexpc(A, b, t0, rmax = 10000)
    p4 = a * tmp

    P = p1 + p2 + ccdf(dist, t0) * (p3 + p4)
    gth(P)
end

In [ ]:
function gbar(t, n, pss, D0, D1)
    a, A, b = GN(n, D0, D1)
    sum((a * exp(Matrix(A)*t) * b)' * pss)
end

In [ ]:
function T(t0, dist)
    res = quadgk(x -> ccdf(dist, x), 0, t0, rtol=1e-8)
    res[1]
end

In [ ]:
function C(t0, dist, cp, cf)
    cp + (cf - cp)*cdf(dist, t0)
end

In [ ]:
# standard age replacement
As(t0, dist, cp, cf) = C(t0, dist, cp, cf) / T(t0, dist)

In [ ]:
function Top(t0, n, dist, pss, D0, D1)
    a, A, b = GN(n, D0, D1)
    tmp = mexpmix(A, b, bounds = (0.0, t0), rmax = 10000) do t
        ccdf(dist, t)
    end
    sum((a * tmp)' * pss)
end

In [ ]:
function Cop(t0, n, dist, co, cp, cf, pss, D0, D1)
    a, A, b = GN(n, D0, D1)
    tmp = mexpmix(A, b, bounds = (0.0, t0), rmax = 10000) do t
        pdf(dist, t)
    end
    t1 = sum((a * tmp)' * pss)
    tmp = mexp(A, b, t0, rmax = 10000)
    t2 = sum((a * tmp)' * pss)
    cp + (cf - co) * t1 - (cp - co) * (1 - t2 * ccdf(dist, t0))
end

In [ ]:
# opportunity
function Aop(t0, n, dist, co, cp, cf, D0, D1)
    pss = PfProb(t0, n, D0, D1, dist)
    Cop(t0, n, dist, co, cp, cf, pss, D0, D1) / Top(t0, n, dist, pss, D0, D1)
end

In [ ]:
# opportunity
function Aop2(t0, n, dist, co, cp, cf, init, D0, D1)
    Cop(t0, n, dist, co, cp, cf, init, D0, D1) / Top(t0, n, dist, init, D0, D1)
end

In [ ]:
function failure_param(;mttf, cv)
    m = optimize(m -> (gamma(1+2/m)/gamma(1+1/m)^2 - 1.0 - cv^2)^2, 0.0001, 10.0).minimizer
    eta = mttf / gamma(1+1/m)
    dist = Weibull(m, eta)

    println("failure shape parameter ", m)
    println("failure scale parameter ", eta)
    println("failure mttf ", mttf)
    println("failure cv ", cv)
    return dist
end

In [ ]:
function poisson_param(;mttop)
    # poisson
    D0 = Float64[-2 1; 1 -2]
    D1 = Float64[1 0; 0 1]

    ss = gth(Matrix(D0+D1))
    tmp = (-D0)' \ ss
    mo = sum(tmp)

    # scale
    D0 = D0 .* mo ./ mttop
    D1 = D1 .* mo ./ mttop

    ss = gth(Matrix(D0+D1))
    tmp = (-D0)' \ ss
    mo = sum(tmp)
    tmp = (-D0)' \ tmp
    mo2 = 2*sum(tmp)

    println("mean opportunity interval ", mo)
    println("cv for opportunity interval ", sqrt(mo2 - mo^2)/mo)
    return D0, D1, 1.0
end

function hyporenewal_param(;mttop, b)
    # hypo renewal
#     b = Float64[1, 1]
    D0 = Float64[-b[1] b[1]; 0 -b[2]]
    D1 = Float64[0 0; b[2] 0]

    ss = gth(Matrix(D0+D1))
    tmp = (-D0)' \ ss
    mo = sum(tmp)

    # scale
    D0 = D0 .* mo ./ mttop
    D1 = D1 .* mo ./ mttop

    ss = gth(Matrix(D0+D1))
    tmp = (-D0)' \ ss
    mo = sum(tmp)
    tmp = (-D0)' \ tmp
    mo2 = 2*sum(tmp)
    cv = sqrt(mo2 - mo^2)/mo
    println("mean opportunity interval ", mo)
    println("cv for opportunity interval ", cv)
    return D0, D1, cv
end

function hyperrenewal_param(;mttop, u, b)
    # hyper exp
    D0 = Float64[-b[1] 0; 0 -b[2]]
    D1 = reshape(b, 2, 1) * reshape(u, 1, 2)

    ss = gth(Matrix(D0+D1))
    tmp = (-D0)' \ ss
    mo = sum(tmp)

    # scale
    D0 = D0 .* mo ./ mttop
    D1 = D1 .* mo ./ mttop

    ss = gth(Matrix(D0+D1))
    tmp = (-D0)' \ ss
    mo = sum(tmp)
    tmp = (-D0)' \ tmp
    mo2 = 2*sum(tmp)
    cv = sqrt(mo2 - mo^2)/mo
    println("mean opportunity interval ", mo)
    println("cv for opportunity interval ", cv)
    return D0, D1, cv

end

function mmpp_param(;mttop, u, b)
    # mmpp
    D0 = Float64[-(u[1]+b[1]) u[1]; u[2] -(u[2]+b[2])]
    D1 = Float64[b[1] 0; 0 b[2]]

    ss = gth(Matrix(D0+D1))
    tmp = (-D0)' \ ss
    mo = sum(tmp)

    # scale
    D0 = D0 .* mo ./ mttop
    D1 = D1 .* mo ./ mttop

    ss = gth(Matrix(D0+D1))
    tmp = (-D0)' \ ss
    mo = sum(tmp)
    tmp = (-D0)' \ tmp
    mo2 = 2*sum(tmp)

    cv = sqrt(mo2 - mo^2)/mo
    println("mean opportunity interval ", mo)
    println("cv for opportunity interval ", cv)
    return D0, D1, cv
end

In [ ]:
function optim(;maxn = 30, mint=1.0, maxt=mttf*2.0, dist, co, cp, cf, D0, D1)
    local minvalue = 100.0
    local minn = 0
    local minresult
    for n = 1:maxn
        result = optimize(x -> Aop(x, n, dist, co, cp, cf, D0, D1), mint, maxt)
        if result.minimum <= minvalue
            minvalue = result.minimum
            minresult = result
            minn = n
        end
    end
    res = optimize(t0 -> As(t0, dist, cp, cf), mint, maxt)
    if res.minimum < minresult.minimum
        "infinity", res
    else
        minn, minresult
    end
end

In [ ]:
function optim2(;maxn = 50, mint=1.0, maxt=mttf*2.0, dist, co, cp, cf, init, D0, D1)
    local minvalue = 100.0
    local minn = 0
    local minresult
    for n = 1:maxn
        result = optimize(x -> Aop2(x, n, dist, co, cp, cf, init, D0, D1), mint, maxt)
        if result.minimum <= minvalue
            minvalue = result.minimum
            minresult = result
            minn = n
        end
    end
    res = optimize(t0 -> As(t0, dist, cp, cf), mint, maxt)
    if res.minimum < minresult.minimum
        "infinity", res
    else
        minn, minresult
    end
end

In [ ]:
co, cp, cf = 1.0, 1.0, 10.0
mttf = 10.0
dist = failure_param(mttf = mttf, cv = 0.1)
D0, D1 = hyporenewal_param(mttop = 1.0, b=[1,2])

In [ ]:
ts = LinRange(1, 20.0, 100)
value = [As(t0, dist, cp, cf) for t0 = ts]
value1 = [Aop(t0, 27, dist, co, cp, cf, D0, D1) for t0 = ts];
plot(ts, value)
plot!(ts, value1)

In [ ]:
cp = 1.0
cf = 10.0
mttf = 10.0
result = []
for cv in [0.1, 0.5, 0.8]
    println("cv ", cv)
    mttop = 0
    ocv = 0
    dist = failure_param(mttf = mttf, cv = cv)
    result0 = optimize(t0 -> As(t0, dist, cp, cf), 1, mttf*2)
    push!(result, ("st", cp, cv, mttop, ocv, 0, result0.minimizer, result0.minimum, result0.converged))
    for co in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
        println("co ", co)
        for mttop in [0.1, 1.0, 2.0, 5.0, 10.0]
            println("mttop ", mttop)
            D0, D1, ocv = poisson_param(mttop = mttop)
            opn, result1 = optim(dist=dist, co=co, cp=cp, cf=cf, D0=D0, D1=D1)
            push!(result, ("poi", co, cv, mttop, ocv, opn, result1.minimizer, result1.minimum, result1.converged))
            for b in [[1, 5], [1, 2], [1, 1]]
                D0, D1, ocv = hyporenewal_param(mttop = mttop, b=b)
                opn, result1 = optim(dist=dist, co=co, cp=cp, cf=cf, D0=D0, D1=D1)
                push!(result, ("hypo", co, cv, mttop, ocv, opn, result1.minimizer, result1.minimum, result1.converged))
            end
            b = [10, 1]
            for u in [[0.2, 0.8], [0.5, 0.5], [0.8, 0.2]]
                D0, D1, ocv = hyperrenewal_param(mttop = mttop, u=u, b=b)
                opn, result1 = optim(dist=dist, co=co, cp=cp, cf=cf, D0=D0, D1=D1)
                push!(result, ("hyper", co, cv, mttop, ocv, opn, result1.minimizer, result1.minimum, result1.converged))
            end
            u = [1, 1]
            for b in [[1, 2], [1, 5], [1, 10]]
                D0, D1, ocv = mmpp_param(mttop = mttop, u=u, b=b)
                opn, result1 = optim(dist=dist, co=co, cp=cp, cf=cf, D0=D0, D1=D1)
                push!(result, ("mmpp", co, cv, mttop, ocv, opn, result1.minimizer, result1.minimum, result1.converged))
            end
        end
    end
end
result = DataFrame(result)
@show result
result |> CSV.write("rf.csv", delim=',', header=false)

In [ ]:
cp = 1.0
cf = 10.0
mttf = 10.0
result = []
for cv in [0.1] # [0.1, 0.5, 0.8]
    println("cv ", cv)
    mttop = 0
    ocv = 0
    dist = failure_param(mttf = mttf, cv = cv)
#     result0 = optimize(t0 -> As(t0, dist, cp, cf), 1, mttf*2)
#     push!(result, ("st", cp, cv, mttop, ocv, 0, result0.minimizer, result0.minimum, result0.converged))
    for co in [0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
        println("co ", co)
        for mttop in [1.0] # [0.1, 1.0, 2.0, 5.0, 10.0]
            println("mttop ", mttop)
#             D0, D1, ocv = poisson_param(mttop = mttop)
#             opn, result1 = optim(dist=dist, co=co, cp=cp, cf=cf, D0=D0, D1=D1)
#             push!(result, ("poi", co, cv, mttop, ocv, opn, result1.minimizer, result1.minimum, result1.converged))
            for b in [[1, 1], [1, 2]] # [[1, 5], [1, 2], [1, 1]]
                D0, D1, ocv = hyporenewal_param(mttop = mttop, b=b)
                init = Float64[1,0]
                opn, result1 = optim2(dist=dist, co=co, cp=cp, cf=cf, init=init, D0=D0, D1=D1)
                push!(result, ("hypo", co, cv, mttop, ocv, opn, result1.minimizer, result1.minimum, result1.converged))
            end
            b = [10, 1]
            for u in [[0.5, 0.5], [0.8, 0.2]] # [[0.2, 0.8], [0.5, 0.5], [0.8, 0.2]]
                D0, D1, ocv = hyperrenewal_param(mttop = mttop, u=u, b=b)
                init = u
                opn, result1 = optim2(dist=dist, co=co, cp=cp, cf=cf, init=init, D0=D0, D1=D1)
                push!(result, ("hyper", co, cv, mttop, ocv, opn, result1.minimizer, result1.minimum, result1.converged))
            end
#             u = [1, 1]
#             for b in [[1, 2], [1, 5], [1, 10]]
#                 D0, D1, ocv = mmpp_param(mttop = mttop, u=u, b=b)
#                 init = Float64[0.5,0.5]
#                 opn, result1 = optim2(dist=dist, co=co, cp=cp, cf=cf, init=init, D0=D0, D1=D1)
#                 push!(result, ("mmpp", co, cv, mttop, ocv, opn, result1.minimizer, result1.minimum, result1.converged))
#             end
        end
    end
end
result = DataFrame(result)
@show result
result |> CSV.write("rf2.csv", delim=',', header=false)

In [ ]:
# check
cp = 1.0
cf = 1.2
co = 0.8
mttf = 10.0
cv = 0.1
mttop = 1.0
dist = failure_param(mttf = mttf, cv = cv)
# D0, D1, ocv = poisson_param(mttop = mttop)
# D0, D1, ocv = hyporenewal_param(mttop = mttop, b=[1,2]) # [[1, 5], [1, 2], [1, 1]]
# D0, D1, ocv = mmpp_param(mttop = mttop, u=[1,1], b=[1,2]) # u=[1,1] b=[[1, 2], [1, 5], [1, 10]]
D0, D1, ocv = hyperrenewal_param(mttop = mttop, u=[0.2,0.8], b=[10,1]) # b = [10, 1], u = [[0.2, 0.8], [0.5, 0.5], [0.8, 0.2]]
opn, result1 = optim(dist=dist, co=co, cp=cp, cf=cf, D0=D0, D1=D1, maxn=40, maxt=20.0)
println(opn, result1)

In [ ]:
ts = LinRange(15, 25.0, 50)
val = [Aop(t0, 6, dist, co, cp, cf, D0, D1) for t0 = ts]
plot(ts, val)
# Aop(20.0, 5, dist, co, cp, cf, D0, D1)